In [1]:
import numpy as np
import cv2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [7]:
train_path = 'train'
test_path = 'test'


train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)


train_generator = train_datagen.flow_from_directory(
                    train_path,
                    target_size = (48,48),
                    batch_size = 64, 
                    color_mode = 'grayscale',
                    class_mode = 'categorical'
                )

test_generator = test_datagen.flow_from_directory(
                    test_path,
                    target_size = (48,48),
                    batch_size = 64, 
                    color_mode = 'grayscale',
                    class_mode = 'categorical'
                )


checkpoint = ModelCheckpoint('model_checkpoint', save_best_only = True, save_weights_only = True, monitor = "val_loss")

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [8]:
model = Sequential()

model.add(Conv2D(32, kernel_size = (3,3), activation = "relu", input_shape= (48,48,1)))
model.add(Conv2D(64, kernel_size = (3,3), activation= "relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(128, kernel_size = (3,3), activation = "relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128, kernel_size = (3,3), activation= "relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(7,activation="softmax"))

In [9]:
model.compile(loss = "categorical_crossentropy" , optimizer = "adam", metrics = ['accuracy'])

model.fit_generator(train_generator, epochs = 20 , steps_per_epoch = 28709//64, validation_data = test_generator, validation_steps = 7178//64, callbacks=[checkpoint]) 

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 448 steps, validate for 112 steps
Epoch 1/20
448/448 [==============================] - 16s 35ms/step - loss: 1.7219 - accuracy: 0.3030 - val_loss: 1.5012 - val_accuracy: 0.4272
Epoch 2/20
448/448 [==============================] - 14s 32ms/step - loss: 1.4134 - accuracy: 0.4529 - val_loss: 1.2877 - val_accuracy: 0.5057
Epoch 3/20
448/448 [==============================] - 14s 31ms/step - loss: 1.2644 - accuracy: 0.5165 - val_loss: 1.1976 - val_accuracy: 0.5426
Epoch 4/20
448/448 [==============================] - 14s 31ms/step - loss: 1.1925 - accuracy: 0.5455 - val_loss: 1.1567 - val_accuracy: 0.5610
Epoch 5/20
448/448 [==============================] - 14s 31ms/step - loss: 1.1375 - accuracy: 0.5677 - val_loss: 1.1335 - val_accuracy: 0.5663
Epoch 6/20
448/448 [==============================] - 14s 31ms/step - loss: 1.0882 - accuracy: 0.5880 - val_loss: 1.1153 - val_accuracy: 0.5752
Epoch 7/20
448/448 [======================

In [10]:
model.save_weights('model.h5')

In [12]:
cv2.ocl.setUseOpenCL(False)

emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

cap = cv2.VideoCapture(0)
while(True):
    ret, frame = cap.read()
    
    if ret is False:
        break
    bounding_box = cv2.CascadeClassifier(r'C:\Users\garim\.conda\haarcascade_frontalface_default.xml')        
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2gray_frame)
    num_faces = bounding_box.detectMultiScale(gray_frame,1.3,5)
    
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Video', cv2.resize(frame,(1200,860),interpolation = cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()
    